In [3]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

h2o.init()

# Import a sample binary outcome train/test set into H2O
#train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
#test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

train = pd.read_csv('C:/Users/n0269042/Documents/h2o/higgs_train_10k.csv')
test = pd.read_csv('C:/Users/n0269042/Documents/h2o/higgs_test_5k.csv')

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 30)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 12 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.4
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_n0269042_c1dwk2
H2O cluster total nodes:,1
H2O cluster free memory:,26.30 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
StackedEnsemble_AllModels_0_AutoML_20180427_141351,0.783536,0.558155
StackedEnsemble_BestOfFamily_0_AutoML_20180427_141351,0.781232,0.560777
GBM_grid_0_AutoML_20180427_141351_model_1,0.774555,0.566055
GBM_grid_0_AutoML_20180427_141351_model_0,0.772745,0.567602
GBM_grid_0_AutoML_20180427_141351_model_2,0.771832,0.573173
DRF_0_AutoML_20180427_141351,0.743365,0.595192
XRT_0_AutoML_20180427_141351,0.739538,0.60507
GLM_grid_0_AutoML_20180427_141351_model_0,0.685543,0.635206


In [5]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model
import h2o
from h2o.automl import H2OAutoML

h2o.init()
import os
os.chdir('C:/Users/n0269042/Documents/numerai')

training_data = pd.read_csv('data/numerai_20180423/numerai_training_data.csv', header=0)
prediction_data = pd.read_csv('data/numerai_20180423/numerai_tournament_data.csv', header=0)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [104]:
train = training_data[training_data.era.isin(['era'+str(i+1) for i in range(100)])].drop(['id', 'era', 'data_type'], axis=1)
test = training_data[training_data.era.isin(['era'+str(i+41) for i in range(20)])].drop(['id', 'era', 'data_type'], axis=1)
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [105]:
x = train.columns
y = "target"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 18000)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
DRF_0_AutoML_20180427_175928,0.942642,0.569917
XRT_0_AutoML_20180427_175928,0.93983,0.574907
GBM_grid_0_AutoML_20180427_175928_model_4,0.720417,0.661013
GBM_grid_0_AutoML_20180427_175928_model_3,0.667048,0.672532
GBM_grid_0_AutoML_20180427_175928_model_6,0.652268,0.689059
StackedEnsemble_BestOfFamily_0_AutoML_20180427_175928,0.645094,0.681878
StackedEnsemble_AllModels_0_AutoML_20180427_175928,0.610974,0.684567
GBM_grid_0_AutoML_20180427_175928_model_2,0.602367,0.683481
GBM_grid_0_AutoML_20180427_175928_model_1,0.58169,0.686391
GBM_grid_0_AutoML_20180427_175928_model_0,0.565247,0.688302


In [106]:
preds = aml.predict(test)
from sklearn.metrics import log_loss
preds = preds.drop(['predict'])
log_loss(test['target'].as_data_frame().values, preds.as_data_frame().values)

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


0.56991725306741603

In [107]:
holdout = h2o.H2OFrame(prediction_data.drop(['id', 'era', 'data_type', 'target'], axis=1))
preds = aml.predict(holdout)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [113]:
test_submission = pd.DataFrame({'id': prediction_data.id, 'probability': preds.as_data_frame().p1})

In [115]:
test_submission.to_csv('submission/test_submission_20180430.csv', index=False)

In [ ]:
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=train)

## Show a detailed model summary
print gbm